In [1]:
import torch
from botorch import acquisition, settings
from utils.optimizer import BayesianOptimizer,LGSO
from utils import problems
from matplotlib import pyplot as plt
import argparse
from utils.models import GP_RBF, GP_Cylindrical_Custom
import botorch

In [2]:
dev = torch.device('cuda')

In [3]:
INITIAL_PHI = torch.tensor([[208.0, 207.0, 281.0, 248.0, 305.0, 242.0, 72.0, 51.0, 29.0, 46.0, 10.0, 7.0, 54.0,
                         38.0, 46.0, 192.0, 14.0, 9.0, 10.0, 31.0, 35.0, 31.0, 51.0, 11.0, 3.0, 32.0, 54.0, 
                         24.0, 8.0, 8.0, 22.0, 32.0, 209.0, 35.0, 8.0, 13.0, 33.0, 77.0, 85.0, 241.0, 9.0, 26.0]],device = dev)

In [4]:
problem_fn = problems.ShipMuonShield()
dimensions_phi = INITIAL_PHI.size(-1)
bounds = problem_fn.GetBounds(device=dev)
model = GP_RBF(bounds,dev)
acquisition_fn = botorch.acquisition.ExpectedImprovement
acquisition_params = {'num_restarts': 20, 'raw_samples':2000}


In [5]:
from time import time
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
times_gpu = []
n_list = [1,10,50,500,750,1000,2500,5000,7500,10000,12500,15000]
N = 50
for N_initial_points in n_list:
    print(N_initial_points)
    t = 0
    for _ in range(N):
        y = torch.randn(N_initial_points,1,device=dev)+249897
        phi = torch.randn(N_initial_points,dimensions_phi,device=dev)+INITIAL_PHI
        start = time()
        model.fit(phi,y)
        acquisition = acquisition_fn(model, 54535, maximize=False)
        botorch.optim.optimize.optimize_acqf(acquisition, bounds, q=1,**acquisition_params)[0]
        t += time()-start
    times_gpu.append(t/N)

In [ ]:
dev = torch.device('cpu')
times_cpu = []
for N_initial_points in n_list:
    print(N_initial_points)
    t = 0
    for _ in range(N):
        y = torch.randn(N_initial_points,1,device=dev)+249897
        phi = torch.randn(N_initial_points,dimensions_phi,device=dev)+INITIAL_PHI
        start = time()
        model.fit(phi,y)
        acquisition = acquisition_fn(model, 54535, maximize=False)
        botorch.optim.optimize.optimize_acqf(acquisition, bounds, q=1,**acquisition_params)[0]
        t += time()-start
    times_cpu.append(t/N)

In [ ]:
plt.plot(n_list,times_gpu, label = 'gpu')
plt.plot(n_list,times_cpu, label = 'cpu')
plt.legend()
plt.grid()
plt.show()